In [1]:
#!/usr/bin/env python

import wandb

from dawgz import job, schedule
from typing import *

from sda.mcs import *
from sda.score import *
from sda.utils import *

from utils import *

In [2]:

CONFIG = {
    # Architecture
    'window': 5,
    'embedding': 32,
    'hidden_channels': (4, 128, 256),
    'hidden_blocks': (3, 3, 3),
    'kernel_size': 3,
    'activation': 'SiLU',
    # Training
    'epochs': 1,
    'batch_size': 5,
    'optimizer': 'AdamW',
    'learning_rate': 2e-4,
    'weight_decay': 1e-3,
    'scheduler': 'linear',
    'size':256,
}


In [3]:



# @job(array=3, cpus=4, gpus=1, ram='64GB', time='4:00:00', account='nvr_earth2_e2', partition='grizzly')
# def train(i: int):
    # run = wandb.init(project='sda-kolmogorov', config=CONFIG)
# runpath = PATH / f'runs/{run.name}_{run.id}'
# runpath.mkdir(parents=True, exist_ok=True)
print(PATH)
# save_config(CONFIG, runpath)

# Network
window = CONFIG['window']
score = make_score(**CONFIG)
sde = VPSDE(score.kernel, shape=(window * 2,256,256)).cuda()



/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov


In [4]:
sde.shape

(10, 256, 256)

In [5]:
PATH

PosixPath('/lustre/fsw/portfolios/nvr/projects/nvr_earth2_e2/sda/kolmogorov')

In [9]:
# # Data

trainset = TrajectoryDataset(PATH / 'data/train.h5', window=window, flatten=True)
validset = TrajectoryDataset(PATH / 'data/valid.h5', window=window, flatten=True)


In [7]:

trainset = TrajectoryDataset( 'data/train.h5', window=window, flatten=True)
validset = TrajectoryDataset('data/valid.h5', window=window, flatten=True)


In [10]:
trainset.__getitem__(7)[0].shape

torch.Size([10, 256, 256])

In [11]:

generator = loop(
    sde,
    trainset,
    validset,
    device='cuda',
    **CONFIG,
)

In [12]:

for loss_train, loss_valid, lr in generator:
    print({
        'loss_train': loss_train,
        'loss_valid': loss_valid,
        'lr': lr,
    })


100%|████████████████████████| 1/1 [00:13<00:00, 13.72s/it, lr=0.0002, lt=1.32, lv=1.12]

{'loss_train': 1.3187837600708008, 'loss_valid': 1.1173171997070312, 'lr': 0.0002}
